# Decision Trees in Practice

In this assignment we will explore various techniques for preventing overfitting in decision trees. We will extend the implementation of the binary decision trees that we implemented in the previous assignment. You will have to use your solutions from this previous assignment and extend them.

In this assignment you will:

* Implement binary decision trees with different early stopping methods.
* Compare models with different stopping parameters.
* Visualize the concept of overfitting in decision trees.

Let's get started!

# Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
import graphlab
print "libraries loaded"

libraries loaded


# Load LendingClub Dataset

This assignment will use the [LendingClub](https://www.lendingclub.com/) dataset used in the previous two assignments.

In [2]:
loans = graphlab.SFrame('../data/lending-club-data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to bmatthewtaylor@gmail.com and will expire on October 27, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\username\AppData\Local\Temp\graphlab_server_1485727897.log.0


In [3]:
print loans.shape
print loans.column_names()

(122607, 68)
['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans', 'emp_length_num', 'grade_num', 'sub_grade_num', 'delinq_2yrs_zero', 'pub_rec_zero', 'collections_12_mths_zero', 'short_emp'

As before, we reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.

In [4]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
#NB: this is _MUCH_ faster than explicit for loop.
print loans.shape
loans = loans.remove_column('bad_loans')
print loans.shape
print loans['safe_loans']

(122607, 69)
(122607, 68)
[1L, -1L, 1L, 1L, 1L, 1L, -1L, -1L, 1L, 1L, -1L, 1L, -1L, 1L, 1L, 1L, 1L, 1L, -1L, 1L, 1L, -1L, 1L, -1L, -1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, -1L, 1L, 1L, 1L, -1L, 1L, 1L, -1L, 1L, -1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, -1L, 1L, -1L, 1L, 1L, -1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, 1L, -1L, 1L, -1L, 1L, 1L, 1L, -1L, 1L, 1L, 1L, 1L, 1L, 1L, ... ]


We will be using the same 4 categorical features as in the previous assignment: 
1. grade of the loan 
2. the length of the loan term
3. the home ownership status: own, mortgage, rent
4. number of years of employment.

In the dataset, each of these features is a categorical feature. Since we are building a binary decision tree, we will have to convert this to binary data in a subsequent section using 1-hot encoding.

In [5]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]
print loans.shape
print loans.column_names()

(122607, 5)
['grade', 'term', 'home_ownership', 'emp_length', 'safe_loans']


In [6]:
print loans.head(5)

+-------+------------+----------------+------------+------------+
| grade |    term    | home_ownership | emp_length | safe_loans |
+-------+------------+----------------+------------+------------+
|   B   |  36 months |      RENT      | 10+ years  |     1      |
|   C   |  60 months |      RENT      |  < 1 year  |     -1     |
|   C   |  36 months |      RENT      | 10+ years  |     1      |
|   C   |  36 months |      RENT      | 10+ years  |     1      |
|   A   |  36 months |      RENT      |  3 years   |     1      |
+-------+------------+----------------+------------+------------+
[5 rows x 5 columns]



## Subsample dataset to make sure classes are balanced

Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We used `seed = 1` so everyone gets the same results.

In [7]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]
print "safe_loans_raw.shape = ", safe_loans_raw.shape
print "risky_loans_raw.shape = ", risky_loans_raw.shape

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)

safe_loans_raw.shape =  (99457, 5)
risky_loans_raw.shape =  (23150, 5)
Percentage of safe loans                 : 0.502236174422
Percentage of risky loans                : 0.497763825578
Total number of loans in our new dataset : 46508


**Note:** There are many approaches for dealing with imbalanced data, including some where we modify the learning algorithm. These approaches are beyond the scope of this course, but some of them are reviewed in this [paper](http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=5128907&url=http%3A%2F%2Fieeexplore.ieee.org%2Fiel5%2F69%2F5173046%2F05128907.pdf%3Farnumber%3D5128907 ). For this assignment, we use the simplest possible approach, where we subsample the overly represented class to get a more balanced dataset. In general, and especially when the data is highly imbalanced, we recommend using more advanced methods.

## Transform categorical data into binary features

Since we are implementing **binary decision trees**, we transform our categorical data into binary data using 1-hot encoding, just as in the previous assignment. Here is the summary of that discussion:

For instance, the **home_ownership** feature represents the home ownership status of the loanee, which is either `own`, `mortgage` or `rent`. For example, if a data point has the feature 
```
   {'home_ownership': 'RENT'}
```
we want to turn this into three features: 
```
 { 
   'home_ownership = OWN'      : 0, 
   'home_ownership = MORTGAGE' : 0, 
   'home_ownership = RENT'     : 1
 }
```

Since this code requires a few Python and GraphLab tricks, feel free to use this block of code as is. Refer to the API documentation for a deeper understanding.


In [25]:
print "'original' loans_data.shape = ", loans_data.shape
print "risky_loans.shape = ", risky_loans.shape
print "safe_loans.shape = ", safe_loans.shape
loans_data = risky_loans.append(safe_loans)
print "loans_data.shape = ", loans_data.shape
print loans_data.column_names()
print "features = ", features
#NB: features defined above as the list of columns we are interested in.
for feature in features:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})  
    #type(loans_data_one_hot_encoded) = SArray
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature)
    print feature, "\t", "type(loans_data_unpacked) = ", type(loans_data_unpacked), loans_data_unpacked.shape
    #print "type(loans_data[feature]) = ", type(loans_data[feature])#SArray
    #print loans_data[feature]
    print "loans_data[feature].shape = ", loans_data[feature].shape
    print loans_data_unpacked.column_names()
    #https://turi.com/products/create/docs/generated/graphlab.SFrame.unpack.html
    #Expand one column of this SFrame to multiple columns with each value in a separate column. 
    #Returns a new SFrame with the unpacked column replaced with a list of new columns.
    
    # Change None's to 0's
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)

    loans_data.remove_column(feature)
    loans_data.add_columns(loans_data_unpacked)
print loans_data.shape

'original' loans_data.shape =  (46508, 26)
risky_loans.shape =  (23150, 5)
safe_loans.shape =  (23358, 5)
loans_data.shape =  (46508, 5)
['grade', 'term', 'home_ownership', 'emp_length', 'safe_loans']
features =  ['grade', 'term', 'home_ownership', 'emp_length']
grade 	type(loans_data_unpacked) =  <class 'graphlab.data_structures.sframe.SFrame'> (46508, 7)
loans_data[feature].shape =  (46508,)
['grade.A', 'grade.B', 'grade.C', 'grade.D', 'grade.E', 'grade.F', 'grade.G']
term 	type(loans_data_unpacked) =  <class 'graphlab.data_structures.sframe.SFrame'> (46508, 2)
loans_data[feature].shape =  (46508,)
['term. 36 months', 'term. 60 months']
home_ownership 	type(loans_data_unpacked) =  <class 'graphlab.data_structures.sframe.SFrame'> (46508, 4)
loans_data[feature].shape =  (46508,)
['home_ownership.MORTGAGE', 'home_ownership.OTHER', 'home_ownership.OWN', 'home_ownership.RENT']
emp_length 	type(loans_data_unpacked) =  <class 'graphlab.data_structures.sframe.SFrame'> (46508, 12)
loans_data[

In [26]:
print loans_data.shape
print loans_data.head(5)

(46508, 26)
+------------+---------+---------+---------+---------+---------+---------+---------+
| safe_loans | grade.A | grade.B | grade.C | grade.D | grade.E | grade.F | grade.G |
+------------+---------+---------+---------+---------+---------+---------+---------+
|     -1     |    0    |    0    |    1    |    0    |    0    |    0    |    0    |
|     -1     |    0    |    0    |    0    |    0    |    0    |    1    |    0    |
|     -1     |    0    |    1    |    0    |    0    |    0    |    0    |    0    |
|     -1     |    0    |    0    |    1    |    0    |    0    |    0    |    0    |
|     -1     |    0    |    1    |    0    |    0    |    0    |    0    |    0    |
+------------+---------+---------+---------+---------+---------+---------+---------+
+-----------------+-----------------+-------------------------+----------------------+
| term. 36 months | term. 60 months | home_ownership.MORTGAGE | home_ownership.OTHER |
+-----------------+-----------------+------------

The feature columns now look like this:

In [27]:
features = loans_data.column_names()
features.remove('safe_loans')  # Remove the response variable
features

['grade.A',
 'grade.B',
 'grade.C',
 'grade.D',
 'grade.E',
 'grade.F',
 'grade.G',
 'term. 36 months',
 'term. 60 months',
 'home_ownership.MORTGAGE',
 'home_ownership.OTHER',
 'home_ownership.OWN',
 'home_ownership.RENT',
 'emp_length.1 year',
 'emp_length.10+ years',
 'emp_length.2 years',
 'emp_length.3 years',
 'emp_length.4 years',
 'emp_length.5 years',
 'emp_length.6 years',
 'emp_length.7 years',
 'emp_length.8 years',
 'emp_length.9 years',
 'emp_length.< 1 year',
 'emp_length.n/a']

## Train-Validation split

We split the data into a train-validation split with 80% of the data in the training set and 20% of the data in the validation set. We use `seed=1` so that everyone gets the same result.

In [30]:
train_data, validation_set = loans_data.random_split(.8, seed=1)
print "train_data.shape = ", train_data.shape
print "validation_set.shape = ", validation_set.shape
print train_data.shape[0] + validation_set.shape[0]
print "loans_data.shape = ", loans_data.shape

train_data.shape =  (37224, 26)
validation_set.shape =  (9284, 26)
46508
loans_data.shape =  (46508, 26)


# Early stopping methods for decision trees

In this section, we will extend the **binary tree implementation** from the previous assignment in order to handle some early stopping conditions. Recall the 3 early stopping methods that were discussed in lecture:

1. Reached a **maximum depth**. (set by parameter `max_depth`).
2. Reached a **minimum node size**. (set by parameter `min_node_size`).
3. Don't split if the **gain in error reduction** is too small. (set by parameter `min_error_reduction`).

For the rest of this assignment, we will refer to these three as **early stopping conditions 1, 2, and 3**.

## Early stopping condition 1: Maximum depth

Recall that we already implemented the maximum depth stopping condition in the previous assignment. In this assignment, we will experiment with this condition a bit more and also write code to implement the 2nd and 3rd early stopping conditions.

We will be reusing code from the previous assignment and then building upon this.  We will **alert you** when you reach a function that was part of the previous assignment so that you can simply copy and past your previous code.

## Early stopping condition 2: Minimum node size

The function **reached_minimum_node_size** takes 2 arguments:

1. The `data` (from a node)
2. The minimum number of data points that a node is allowed to split on, `min_node_size`.

This function simply calculates whether the number of data points at a given node is less than or equal to the specified minimum node size. This function will be used to detect this early stopping condition in the **decision_tree_create** function.

Fill in the parts of the function below where you find `## YOUR CODE HERE`.  There is **one** instance in the function below.

In [32]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    #NB: could also use data.shape[0]
    #return data.shape[0] < min_node_size  
    return len(data) < min_node_size    

** Quiz Question 1:** Given an intermediate node with 6 safe loans and 3 risky loans, if the `min_node_size` parameter is 10, what should the tree learning algorithm do next?  

6+3=9  
9<10  
stop when node size is less than min_node_size  
ans: create a leaf and return it

## Early stopping condition 3: Minimum gain in error reduction

The function **error_reduction** takes 2 arguments:

1. The error **before** a split, `error_before_split`.
2. The error **after** a split, `error_after_split`.

This function computes the gain in error reduction, i.e., the difference between the error before the split and that after the split. This function will be used to detect this early stopping condition in the **decision_tree_create** function.

Fill in the parts of the function below where you find `## YOUR CODE HERE`.  There is **one** instance in the function below. 

In [35]:
def error_reduction(error_before_split, error_after_split):
    # Return the error before the split minus the error after the split.
    return error_before_split - error_after_split
print "function error_reduction loaded."

function error_reduction loaded.


** Quiz Question 2:** Assume an intermediate node has 6 safe loans and 3 risky loans.  For each of 4 possible features to split on, the error reduction is 0.0, 0.05, 0.1, and 0.14, respectively.   

If the **minimum gain in error reduction** parameter is set to 0.2, what should the tree learning algorithm do next?  


The minimum gain in error reduction (0.2) is > all possible error reduction. [0.0, 0.05, 0.1, 0.14]  

tree learning algorithm should create leaf and STOP
ans: Create a leaf and return it

## Grabbing binary decision tree helper functions from past assignment

Recall from the previous assignment that we wrote a function `intermediate_node_num_mistakes` that calculates the number of **misclassified examples** when predicting the **majority class**. This is used to help determine which feature is best to split on at a given node of the tree.

**Please copy and paste your code for `intermediate_node_num_mistakes` here**.

In [34]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    #labels_in_node : <class 'graphlab.data_structures.sarray.SArray'>
    if len(labels_in_node) == 0:
        return 0    
    # Count the number of 1's (safe loans)
    numSafeLoans = sum(labels_in_node == 1)
    # Count the number of -1's (risky loans)
    numRiskyLoans = sum(labels_in_node == -1)
    # Return the number of mistakes that the majority classifier makes.
    #NB : Since we are assuming majority class prediction, all the data 
    #points that are not in the majority class are considered mistakes.
    return min(numSafeLoans, numRiskyLoans)

print "function intermediate_node_num_mistakes loaded"

function intermediate_node_num_mistakes loaded


We then wrote a function `best_splitting_feature` that finds the best feature to split on given the data and a list of features to consider.

**Please copy and paste your `best_splitting_feature` code here**.

In [37]:
def best_splitting_feature(data, features, target):
    '''
    NB: argument types
    data : <class 'graphlab.data_structures.sframe.SFrame'>
    features : <type 'list'>
    target : <type 'string'>
    '''
    print "data : ", type(data), data.shape
    print "features : ", type(features), len(features)
    print "target : ", target
    
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    #NB: why best_error is initialized at = 10. SEE info BELOW!!!
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        # step 2 : Within the loop, split the data into two groups: one group 
        # where all of the data has feature value 0 or False (we will call this the left split),
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        # step 2 : Within the loop, split the data into two groups: 
        # one group where all of the data has feature value 1 or True (we will call this the right split). 
        right_split =  data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        left_mistakes = intermediate_node_num_mistakes(left_split[target])

        # Calculate the number of misclassified examples in the right split.
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        #error = (left_mistakes+right_mistakes)/num_data_points
        error = float(left_mistakes+right_mistakes)/len(data)
        
        #print feature, " : # rows : left split:", left_split.shape[0], " : right_split:", right_split.shape[0]
        #print "left mistakes : ", left_mistakes, ", right_mistakes : ", right_mistakes, ", error = ", error

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_error = error
            best_feature = feature
            
    return best_feature # Return the best feature we found
print "function best_splitting_feature loaded"

function best_splitting_feature loaded


Finally, recall the function `create_leaf` from the previous assignment, which creates a leaf node given a set of target values.  

**Please copy and paste your `create_leaf` code here**.

In [39]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True    }   ## YOUR CODE HERE
    
    # Count the number of data points that are +1 and -1 in _this_ node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = +1        ## YOUR CODE HERE
    else:
        leaf['prediction'] = -1         ## YOUR CODE HERE
        
    # Return the leaf node        
    return leaf 
print "function create_leaf loaded."

function create_leaf loaded.


## Incorporating new early stopping conditions in binary decision tree implementation

Now, you will implement a function that builds a decision tree handling the three early stopping conditions described in this assignment.  In particular, you will write code to detect early stopping conditions 2 and 3.  You implemented above the functions needed to detect these conditions.  The 1st early stopping condition, **max_depth**, was implemented in the previous assigment and you will not need to reimplement this.  In addition to these early stopping conditions, the typical stopping conditions of having no mistakes or no more features to split on (which we denote by "stopping conditions" 1 and 2) are also included as in the previous assignment.

**Implementing early stopping condition 2: minimum node size:**

* **Step 1:** Use the function **reached_minimum_node_size** that you implemented earlier to write an if condition to detect whether we have hit the base case, i.e., the node does not have enough data points and should be turned into a leaf. Don't forget to use the `min_node_size` argument.
* **Step 2:** Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.


**Implementing early stopping condition 3: minimum error reduction:**

**Note:** This has to come after finding the best splitting feature so we can calculate the error after splitting in order to calculate the error reduction.

* **Step 1:** Calculate the **classification error before splitting**.  Recall that classification error is defined as:

$$
\text{classification error} = \frac{\text{# mistakes}}{\text{# total examples}}
$$
* **Step 2:** Calculate the **classification error after splitting**. This requires calculating the number of mistakes in the left and right splits, and then dividing by the total number of examples.
* **Step 3:** Use the function **error_reduction** to that you implemented earlier to write an if condition to detect whether  the reduction in error is less than the constant provided (`min_error_reduction`). Don't forget to use that argument.
* **Step 4:** Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.

Fill in the places where you find `## YOUR CODE HERE`. There are **seven** places in this function for you to fill in.

In [40]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    
    
    # Stopping condition 1: All nodes are of the same type.
    if intermediate_node_num_mistakes(target_values) == 0:
        print "Stopping condition 1 reached. All data points have the same target value."                
        return create_leaf(target_values)
    
    # Stopping condition 2: No more features to split on.
    if remaining_features == []:
        print "Stopping condition 2 reached. No remaining features."                
        return create_leaf(target_values)    
    
    # Early stopping condition 1: Reached max depth limit.
    if current_depth >= max_depth:
        print "Early stopping condition 1 reached. Reached maximum depth."
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    #reached_minimum_node_size(data, min_node_size)
    if  reached_minimum_node_size(data, min_node_size):         
        print "Early stopping condition 2 reached. Reached minimum node size."
        return create_leaf(target_values)  
    
    # Find the best splitting feature
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    #
    #refer function best_splitting_feature for example code.
    left_mistakes = intermediate_node_num_mistakes(left_split[target]) 
    right_mistakes = intermediate_node_num_mistakes(right_split[target]) 
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if error_reduction(error_before_split, error_after_split) <= min_error_reduction:
        print "Early stopping condition 3 reached. Minimum error reduction."
        return create_leaf(target_values)
    
    remaining_features.remove(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split))
    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)
    
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}
print "function decision_tree_create loaded"

function decision_tree_create loaded


Here is a function to count the nodes in your tree:

In [42]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])
print "function count_nodes loaded."

function count_nodes loaded.


Run the following test code to check your implementation. Make sure you get **'Test passed'** before proceeding.

In [43]:
small_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 2, 
                                        min_node_size = 10, min_error_reduction=0.0)
if count_nodes(small_decision_tree) == 7:
    print 'Test passed!'
else:
    print 'Test failed... try again!'
    print 'Number of nodes found                :', count_nodes(small_decision_tree)
    print 'Number of nodes that should be there : 7' 

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (37224, 26)
features :  <type 'list'> 25
target :  safe_loans
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9223, 26)
features :  <type 'list'> 24
target :  safe_loans
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
data :  <class 'gr

## Build a tree!

Now that your code is working, we will train a tree model on the **train_data** with
* `max_depth = 6`
* `min_node_size = 100`, 
* `min_error_reduction = 0.0`

**Warning**: This code block may take a minute to learn. 

In [44]:
my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (37224, 26)
features :  <type 'list'> 25
target :  safe_loans
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9223, 26)
features :  <type 'list'> 24
target :  safe_loans
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9122, 26)
features :  <type 'list'> 23
target :  safe_loans
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (101, 26)

Let's now train a tree model **ignoring early stopping conditions 2 and 3** so that we get the same tree as in the previous assignment.  To ignore these conditions, we set `min_node_size=0` and `min_error_reduction=-1` (a negative value).

In [45]:
my_decision_tree_old = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (37224, 26)
features :  <type 'list'> 25
target :  safe_loans
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9223, 26)
features :  <type 'list'> 24
target :  safe_loans
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9122, 26)
features :  <type 'list'> 23
target :  safe_loans
Split on feature grade.B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (8074, 26)
features :  <type '

## Making predictions

Recall that in the previous assignment you implemented a function `classify` to classify a new point `x` using a given `tree`.

**Please copy and paste your `classify` code here**.

In [46]:
def classify(tree, x, annotate = False):   
    '''
    tree     [<type 'dict'>]
    x        [<type 'dict'>]
    annotate [boolean]
    '''
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction']
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
            print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            if annotate:
                print "left split"
            return classify(tree['left'], x, annotate)
        else:
            if annotate:
                print "right split"                    
            return classify(tree['right'], x, annotate)  
print "function classify loaded."

function classify loaded.


Now, let's consider the first example of the validation set and see what the `my_decision_tree_new` model predicts for this data point.

In [47]:
validation_set[0]

{'emp_length.1 year': 0L,
 'emp_length.10+ years': 0L,
 'emp_length.2 years': 1L,
 'emp_length.3 years': 0L,
 'emp_length.4 years': 0L,
 'emp_length.5 years': 0L,
 'emp_length.6 years': 0L,
 'emp_length.7 years': 0L,
 'emp_length.8 years': 0L,
 'emp_length.9 years': 0L,
 'emp_length.< 1 year': 0L,
 'emp_length.n/a': 0L,
 'grade.A': 0L,
 'grade.B': 0L,
 'grade.C': 0L,
 'grade.D': 1L,
 'grade.E': 0L,
 'grade.F': 0L,
 'grade.G': 0L,
 'home_ownership.MORTGAGE': 0L,
 'home_ownership.OTHER': 0L,
 'home_ownership.OWN': 0L,
 'home_ownership.RENT': 1L,
 'safe_loans': -1L,
 'term. 36 months': 0L,
 'term. 60 months': 1L}

In [48]:
print 'Predicted class: %s ' % classify(my_decision_tree_new, validation_set[0])

Predicted class: -1 


Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [49]:
classify(my_decision_tree_new, validation_set[0], annotate = True)

Split on term. 36 months = 0
left split
Split on grade.A = 0
left split
At leaf, predicting -1


-1

Let's now recall the prediction path for the decision tree learned in the previous assignment, which we recreated here as `my_decision_tree_old`.

In [50]:
classify(my_decision_tree_old, validation_set[0], annotate = True)

Split on term. 36 months = 0
left split
Split on grade.A = 0
left split
Split on grade.B = 0
left split
Split on grade.C = 0
left split
Split on grade.D = 1
right split
Split on grade.E = 0
left split
At leaf, predicting -1


-1

** Quiz Question 3:** For `my_decision_tree_new` trained with   
`max_depth = 6`,   
`min_node_size = 100`,   
`min_error_reduction=0.0`,   
is the prediction path for `validation_set[0]` shorter, longer, or the same  
as for `my_decision_tree_old` that ignored the early stopping conditions 2 and 3?  

my_decision_tree_new has fewer leafs than my_decision_tree_old when tested by validation_set[0].

**Quiz Question 4:** For `my_decision_tree_new` trained with   
`max_depth = 6`,   
`min_node_size = 100`,   
`min_error_reduction=0.0`,   

is the prediction path for **any point**   
__always shorter__,   
__always longer__,   
__always the same__,   
__shorter__ or the same,   
or   
__longer or the same   
as for  `my_decision_tree_old` that ignored the early stopping conditions 2 and 3?

logically
- cannot be longer  
- will not always be the same.  
- cannot be always longer or the same  
- can be short, but won't always be shorter.  

only option left  
will always be shorter or the same.

** Quiz Question 5:** For a tree trained on **any** dataset using   
`max_depth = 6`,   
`min_node_size = 100`,   
`min_error_reduction=0.0`,   
what is the maximum number of splits encountered while making a single prediction?  

maximum number of splits = max_depth = 6

## Evaluating the model

Now let us evaluate the model that we have trained. You implemented this evaluation in the function `evaluate_classification_error` from the previous assignment.

**Please copy and paste your `evaluate_classification_error` code here**.

In [52]:
def evaluate_classification_error(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x))
    #print "type(prediction) =", type(prediction)
    #print "prediction.shape = ", prediction.shape
    #print "type(data[target]) = ", type(data[target])
    #correctResults = prediction == data[target]
    inCorrectResults = prediction != data[target]
    #print "type(correctResults) = ", type(correctResults)
    #print "type(correctResults[0]) = ", type(correctResults[0])
    #print "correctResults[0] = ", correctResults[0]
    #print "sum(correctResults) = ", sum(correctResults)
    #print "len(data) = ", len(data)
    
    # Once you've made the predictions, calculate the classification error and return it
    ## YOUR CODE HERE
    return float(sum(inCorrectResults))/len(data)
print "function evaluate_classification_error loaded."

function evaluate_classification_error loaded.


Now, let's use this function to evaluate the classification error of `my_decision_tree_new` on the **validation_set**.

In [57]:
my_decision_tree_new_error = evaluate_classification_error(my_decision_tree_new, validation_set, target)
print my_decision_tree_new_error

0.383670831538


Now, evaluate the validation error using `my_decision_tree_old`.

In [58]:
my_decision_tree_old_error = evaluate_classification_error(my_decision_tree_old, validation_set, target)
print my_decision_tree_old_error

0.383778543731


In [59]:
print my_decision_tree_new_error - my_decision_tree_old_error

-0.00010771219302


**Quiz Question 6:** Is the validation error of the new decision tree (using early stopping conditions 2 and 3) lower than, higher than, or the same as that of the old decision tree from the previous assignment?  

ANSWER : my_decision_tree_new_error is less than my_decision_tree_old_error

# Exploring the effect of max_depth

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**too small**, **just right**, and **too large**).

Train three models with these parameters:

1. **model_1**: max_depth = 2 (too small)
2. **model_2**: max_depth = 6 (just right)
3. **model_3**: max_depth = 14 (may be too large)

For each of these three, we set `min_node_size = 0` and `min_error_reduction = -1`.

** Note:** Each tree can take up to a few minutes to train. In particular, `model_3` will probably take the longest to train.

In [60]:
model_1 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 2, min_node_size = 0, min_error_reduction=-1)
print type(model_1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (37224, 26)
features :  <type 'list'> 25
target :  safe_loans
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9223, 26)
features :  <type 'list'> 24
target :  safe_loans
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
data :  <class 'gr

In [61]:
model_2 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, min_node_size = 0, min_error_reduction=-1)
print "completed"

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (37224, 26)
features :  <type 'list'> 25
target :  safe_loans
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9223, 26)
features :  <type 'list'> 24
target :  safe_loans
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9122, 26)
features :  <type 'list'> 23
target :  safe_loans
Split on feature grade.B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (8074, 26)
features :  <type '

In [66]:
model_3 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 14, min_node_size = 0, min_error_reduction=-1)
print "completed"

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (37224, 26)
features :  <type 'list'> 25
target :  safe_loans
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9223, 26)
features :  <type 'list'> 24
target :  safe_loans
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9122, 26)
features :  <type 'list'> 23
target :  safe_loans
Split on feature grade.B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (8074, 26)
features :  <type '

### Evaluating the models

Let us evaluate the models on the **train** and **validation** data. Let us start by evaluating the classification error on the training data:

In [69]:
print "--------------"
print "Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data, target)
print "______________"

--------------
Training data, classification error (model 1): 0.400037610144
______________


In [70]:
print "--------------"
print "Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data, target)
print "______________"

--------------
Training data, classification error (model 2): 0.381850419084
______________


In [71]:
print "--------------"
print "Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data, target)
print "______________"

--------------
Training data, classification error (model 3): 0.374462712229
______________


Now evaluate the classification error on the validation data.

In [76]:
print "--------------"
print "Validation data, classification error (model 1):", evaluate_classification_error(model_1, validation_set, target)
print "______________"

--------------
Validation data, classification error (model 1): 0.398104265403
______________


In [77]:
print "--------------"
print "Validation data, classification error (model 2):", evaluate_classification_error(model_2, validation_set, target)
print "______________"

--------------
Validation data, classification error (model 2): 0.383778543731
______________


In [75]:
print "--------------"
print "Validation data, classification error (model 3):", evaluate_classification_error(model_3, validation_set, target)
print "______________"

--------------
Validation data, classification error (model 3): 0.380008616975
______________


**Quiz Question 7:** Which tree has the smallest error on the validation data?  

classification error from validation data   
model 1 = 0.398  
model 2 = 0.3837  
model_3 = 0.380   

model_3 is smallest


**Quiz Question 8:** Does the tree with the smallest error in the training data also have the smallest error in the validation data?

classification error from training data  
model 1 = 0.4000  
model 2 = 0.3818  
model 3 = 0.3744  
  
Validation data, classification error
model 1 = 0.398104265403
model 2 = 0.383778543731
model 3 = 0.380008616975
  
model 3 has smallest classification error on training data and on validation data.
ANSWER : Yes in this case. 


**Quiz Question 9:** Is it always true that the tree with the lowest classification error on the **training** set will result in the lowest classification error in the **validation** set?

**NO**



### Measuring the complexity of the tree

Recall in the lecture that we talked about deeper trees being more complex. We will measure the complexity of the tree as

```
  complexity(T) = number of leaves in the tree T
```

Here, we provide a function `count_leaves` that counts the number of leaves in a tree. Using this implementation, compute the number of nodes in `model_1`, `model_2`, and `model_3`. 

In [79]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])
print "function count_leaves loaded"

function count_leaves loaded


Compute the number of nodes in `model_1`, `model_2`, and `model_3`.

In [81]:
print "number of nodes in model_1 = ", count_leaves(model_1)
print "number of nodes in model_2 = ", count_leaves(model_2)
print "number of nodes in model_3 = ", count_leaves(model_3)

number of nodes in model_1 =  4
number of nodes in model_2 =  41
number of nodes in model_3 =  341


**Quiz Question 10:** Which tree has the largest complexity?

model_3 has the most leaves.

**Quiz Question 11:** Is it always true that the most complex tree will result in the lowest classification error in the **validation_set**?


In this example - the most complex model (ie: has most leaves) has the lowest classifcation error over validation data.
classification error from validation data
model 1 = 0.398  
model 2 = 0.3837  
model_3 = 0.380  

Will NOT always be true.

# Exploring the effect of min_error

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**negative**, **just right**, and **too positive**).

Train three models with these parameters:
1. **model_4**: `min_error_reduction = -1` (ignoring this early stopping condition)
2. **model_5**: `min_error_reduction = 0` (just right)
3. **model_6**: `min_error_reduction = 5` (too positive)

For each of these three, we set `max_depth = 6`, and `min_node_size = 0`.

** Note:** Each tree can take up to 30 seconds to train.

In [84]:
print "start"
model_4 = model_3 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, min_node_size = 0, min_error_reduction=-1)
print "completed"


start
--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (37224, 26)
features :  <type 'list'> 25
target :  safe_loans
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9223, 26)
features :  <type 'list'> 24
target :  safe_loans
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9122, 26)
features :  <type 'list'> 23
target :  safe_loans
Split on feature grade.B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (8074, 26)
features :  <

In [85]:
print "start"
model_5 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, min_node_size = 0, min_error_reduction=0)
print "completed"

start
--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (37224, 26)
features :  <type 'list'> 25
target :  safe_loans
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9223, 26)
features :  <type 'list'> 24
target :  safe_loans
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9122, 26)
features :  <type 'list'> 23
target :  safe_loans
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (10

In [87]:
model_6 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, min_node_size = 0, min_error_reduction=5) 

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (37224, 26)
features :  <type 'list'> 25
target :  safe_loans
Early stopping condition 3 reached. Minimum error reduction.


Calculate the accuracy of each model (**model_4**, **model_5**, or **model_6**) on the validation set. 

In [88]:
print "Validation data, classification error (model 4):", evaluate_classification_error(model_4, validation_set, target)
print "Validation data, classification error (model 5):", evaluate_classification_error(model_5, validation_set, target)
print "Validation data, classification error (model 6):", evaluate_classification_error(model_6, validation_set, target)

Validation data, classification error (model 4): 0.383778543731
Validation data, classification error (model 5): 0.383778543731
Validation data, classification error (model 6): 0.503446790177


Using the `count_leaves` function, compute the number of leaves in each of each models in (**model_4**, **model_5**, and **model_6**). 

In [92]:
print "model_4, min_error_reduction=-1 =  ", count_leaves(model_4)
print "model_5, min_error_reduction=0 = ", count_leaves(model_5)
print "model_6, min_error_reduction=5", count_leaves(model_6)


model_4, min_error_reduction=-1 =   41
model_5, min_error_reduction=0 =  13
model_6, min_error_reduction=5 1


**Quiz Question 12:** Using the complexity definition above, which model (**model_4**, **model_5**, or **model_6**) has the largest complexity?

model 4 has greatest complexity.

Did this match your expectation?

yes. When the impact of reducing the parameter "min_error_reduction" is considered, the impact of smaller min_error_reduction producing more mode and more complexity is espected.

**Quiz Question 13:** **model_4** and **model_5** have similar classification error on the validation set but **model_5** has lower complexity. Should you pick **model_5** over **model_4**?


yes, pick **model_5** over **model_4** because lower complexity is good. Occums Razor etc.

# Exploring the effect of min_node_size

We will compare three models trained with different values of the stopping criterion. Again, intentionally picked models at the extreme ends (**too small**, **just right**, and **just right**).

Train three models with these parameters:
1. **model_7**: min_node_size = 0 (too small)
2. **model_8**: min_node_size = 2000 (just right)
3. **model_9**: min_node_size = 50000 (too large)

For each of these three, we set `max_depth = 6`, and `min_error_reduction = -1`.

** Note:** Each tree can take up to 30 seconds to train.

In [101]:
model_7 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, min_node_size = 0, min_error_reduction=-1)  

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (37224, 26)
features :  <type 'list'> 25
target :  safe_loans
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9223, 26)
features :  <type 'list'> 24
target :  safe_loans
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9122, 26)
features :  <type 'list'> 23
target :  safe_loans
Split on feature grade.B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (8074, 26)
features :  <type '

In [102]:
model_8 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, min_node_size = 2000, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (37224, 26)
features :  <type 'list'> 25
target :  safe_loans
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9223, 26)
features :  <type 'list'> 24
target :  safe_loans
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9122, 26)
features :  <type 'list'> 23
target :  safe_loans
Split on feature grade.B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (8074, 26)
features :  <type '

In [103]:
model_9 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, min_node_size = 50000, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Early stopping condition 2 reached. Reached minimum node size.


In [104]:
print "model_7, min_node_size = 0 (too small) =  ", count_leaves(model_7)
print "model_8, min_node_size = 2000 (just right) = ", count_leaves(model_8)
print "model_9, min_node_size = 50000 (too large)", count_leaves(model_9)

model_7, min_node_size = 0 (too small) =   41
model_8, min_node_size = 2000 (just right) =  19
model_9, min_node_size = 50000 (too large) 1


Now, let us evaluate the models (**model_7**, **model_8**, or **model_9**) on the **validation_set**.

In [105]:
model_7v = decision_tree_create(validation_set, features, 'safe_loans', max_depth = 6, min_node_size = 0, min_error_reduction=-1)  

--------------------------------------------------------------------
Subtree, depth = 0 (9284 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9284, 26)
features :  <type 'list'> 25
target :  safe_loans
Split on feature term. 36 months. (2322, 6962)
--------------------------------------------------------------------
Subtree, depth = 1 (2322 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (2322, 26)
features :  <type 'list'> 24
target :  safe_loans
Split on feature grade.A. (2294, 28)
--------------------------------------------------------------------
Subtree, depth = 2 (2294 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (2294, 26)
features :  <type 'list'> 23
target :  safe_loans
Split on feature grade.B. (2059, 235)
--------------------------------------------------------------------
Subtree, depth = 3 (2059 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (2059, 26)
features :  <type 'list'

In [107]:
model_8v = decision_tree_create(validation_set, features, 'safe_loans', max_depth = 6, min_node_size = 2000, min_error_reduction=-1)

 --------------------------------------------------------------------
Subtree, depth = 0 (9284 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (9284, 26)
features :  <type 'list'> 25
target :  safe_loans
Split on feature term. 36 months. (2322, 6962)
--------------------------------------------------------------------
Subtree, depth = 1 (2322 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (2322, 26)
features :  <type 'list'> 24
target :  safe_loans
Split on feature grade.A. (2294, 28)
--------------------------------------------------------------------
Subtree, depth = 2 (2294 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (2294, 26)
features :  <type 'list'> 23
target :  safe_loans
Split on feature grade.B. (2059, 235)
--------------------------------------------------------------------
Subtree, depth = 3 (2059 data points).
data :  <class 'graphlab.data_structures.sframe.SFrame'> (2059, 26)
features :  <type 'list

In [108]:
model_9v = decision_tree_create(validation_set, features, 'safe_loans', max_depth = 6, min_node_size = 50000, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (9284 data points).
Early stopping condition 2 reached. Reached minimum node size.


Using the `count_leaves` function, compute the number of leaves in each of each models (**model_7**, **model_8**, and **model_9**). 

In [111]:
print "training models"
print "model_7, min_node_size = 0 (too small) =  ", count_leaves(model_7)
print "model_8, min_node_size = 2000 (just right) = ", count_leaves(model_8)
print "model_9, min_node_size = 50000 (too large)", count_leaves(model_9)
print "\n----------------------------------------------------------------------\n"
print "validation models"
print "model_7, min_node_size = 0 (too small) =  ", count_leaves(model_7v)
print "model_8, min_node_size = 2000 (just right) = ", count_leaves(model_8v)
print "model_9, min_node_size = 50000 (too large)", count_leaves(model_9v)

print "\n----------------------------------------------------------------------\n"
print "Validation data, classification error (model 7):", evaluate_classification_error(model_7, validation_set, target)
print "Validation data, classification error (model 8):", evaluate_classification_error(model_8, validation_set, target)
print "Validation data, classification error (model 9):", evaluate_classification_error(model_9, validation_set, target)

training models
model_7, min_node_size = 0 (too small) =   41
model_8, min_node_size = 2000 (just right) =  19
model_9, min_node_size = 50000 (too large) 1

----------------------------------------------------------------------

validation models
model_7, min_node_size = 0 (too small) =   43
model_8, min_node_size = 2000 (just right) =  10
model_9, min_node_size = 50000 (too large) 1

----------------------------------------------------------------------

Validation data, classification error (model 7): 0.383778543731
Validation data, classification error (model 8): 0.384532529082
Validation data, classification error (model 9): 0.503446790177


**Quiz Question 14:** Using the results obtained in this section, which model (**model_7**, **model_8**, or **model_9**) would you choose to use?

In [ ]:
#choose model 8. 